# Migration WorkFlow Example

This is a more complex scenario that describes a workflow where there are two service engine groups that need to be merged into a single one. We will define a **TARGET** service engine group that will be used to receive the configuration extracted from the **SOURCE** service engine group. 

<img src="images/migration_scenario.png">

- [1.- Initial Login](#1--initial-login)
- [2.- Source Service Engine Data Collection](#step-1--collecting-source-service-engine-group-data)

In [ ]:
from envs.controller_info import session_params as session_env
### Import python packages to setup Demo environment.
from libs.migration_scenario_load import setup_scenario
from libs.migration_scenario_clean import clean_scenario


In [ ]:

# Clean Scenario
clean_scenario(session_env)


In [ ]:
setup_scenario(session_env)
    

## Initial Login

In [ ]:
from avi.sdk.avi_api import ApiSession
import datetime, time
from requests.packages import urllib3
urllib3.disable_warnings()
import json
import pandas as pd
from IPython.display import display, Image

# Import environment variables with controller information and credentials
from envs.controller_info import session_params as session_env

# Establish a first session with AVI Controller
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Update headers and api version imported from demo env file with controller version (ensure actual API Version is uses in subsequent requests
session_env['headers']['X-Avi-Version'] = api.remote_api_version['Version']
session_env['api_version'] = api.remote_api_version['Version']

# Create a new session with received AVI API Version
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Display Session ID to Verify AVI Controller Session Establishment
print('Successful connection to ' + session_env['name'] + '. Session ID:' + api.session_id)

In [ ]:
# Set SOURCE AND TARGET Service Engine Group Names
# Source = Group to be migrated 
# Target 
source_segroup = "SEG-SOURCE-MAD-002"
target_segroup = "SEG-TARGET-MAD-001"

## STEP 1.- Collecting Source Service Engine Group Data

The first step is to Gather SE Group Information
- Service Engine Group Configuration
- Service Engine Configuration
- Virtual Services Configuration
- Network Services Configuration (if exists)

In [ ]:
# Gather SOURCE Service Engine Group related configuration
print("\033[1mExtracting SOURCE Service Engine Group Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
print()

# Define GET request parameters
url_path = "serviceenginegroup"
query = {
   "skip_default": "true",
   "name": source_segroup
}
# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
source_segroup_data = resp_data[0]

# Extract related virtual services
# GET Virtual Services Related Configuration
print("\033[1mExtracting SOURCE Virtual Services Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
print()

seg_data = source_segroup_data
# Define GET request parameters
se_group_uuid=seg_data["uuid"]
url_path = "virtualservice"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}"
}

# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
source_vs_data = resp_data

# Extract related Service Engine
print("\033[1mExtracting SOURCE Services Engines information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
print()

# GET Service Engine Related Configuration

# Define GET request parameters
se_group_uuid=seg_data["uuid"]
url_path = "serviceengine"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}"
}

# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
source_se_data = resp_data

## Exploring Source Information

In [ ]:
# Exploring Interface Information of SOURCE SE

for i in range(len(source_se_data)):
    print("\033[1mShowing information of the SOURCE SE "+str(i+1)+" \033[0m")
    print("\033[1m--------------------------------------------------\033[0m")
    se_data_vnics = source_se_data[i]["data_vnics"]
    se_df = pd.DataFrame(se_data_vnics)
    display(se_df)
    time.sleep(1/4)


### Select source Interface to be migrated (ensure you are picking if_name and not linux_name column)


In [ ]:
source_if_name = "eth1"

In [ ]:
# Extract Interface Information from first SE
output_config=[]
for i in range(len(source_se_data)):
    config=[]
    print("\033[1mShowing information of the SOURCE SE "+str(i+1)+" interface to migrate "+source_if_name+" \033[0m")
    print("\033[1m--------------------------------------------------\033[0m")
    se_data_vnics = source_se_data[i]["data_vnics"]
    se_data_vnic_to_migrate = [ adapter for adapter in se_data_vnics if adapter.get("if_name") == source_if_name]
    se_data_vnic_to_migrate_df = pd.DataFrame(se_data_vnic_to_migrate)
    time.sleep(1/4)
    display(se_data_vnic_to_migrate_df)
    se_vnic_to_migrate_config = se_data_vnic_to_migrate[0]["vnic_networks"][0]["ip"]
    se_vnic_to_migrate_config_df = pd.DataFrame(se_vnic_to_migrate_config)
    display(se_vnic_to_migrate_config_df)
    config={"ip_addr":se_data_vnic_to_migrate[0]["vnic_networks"][0]["ip"]["ip_addr"]["addr"],
            "mask": se_data_vnic_to_migrate[0]["vnic_networks"][0]["ip"]["mask"],
            "mac_address": se_data_vnic_to_migrate[0]["mac_address"]}
    output_config.append(config)


In [ ]:
print ("The following interfaces has been selected for migration")
print()
for i in range(len(output_config)):
    print("For the Service Engine "+str(i+1)+" with name \033[1m" + source_se_data[i]["name"]+"\033[0m")
    print("The interface \033[1m" + source_if_name + "\033[0m has been selected with following configuration")
    print("   IP Address \033[1m" + output_config[i]["ip_addr"] +"\033[0m")
    print("   Mask Length \033[1m" + str(output_config[i]["mask"]) +"\033[0m")
    print("   MAC-ADDRESS \033[1m" + output_config[i]["mac_address"] +"\033[0m")
    print()

### TARGET SERVICE ENGINE GROUP

In [ ]:
# GET Service Engine Group Configuration

# Define GET request parameters
# Extract related Service Engine
print("\033[1mExtracting TARGET Services Engine Group information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
print()

url_path = "serviceenginegroup"
query = {
   "skip_default": "true",
   "name": target_segroup
}
# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
target_segroup_data = resp_data[0]

# Extract related virtual services
# GET Virtual Services Related Configuration
print("\033[1mExtracting TARGET Virtual Services Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
print()

seg_data = target_segroup_data
# Define GET request parameters
se_group_uuid=seg_data["uuid"]
url_path = "virtualservice"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}"
}

# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
target_vs_data = resp_data

# Extract related Service Engine

# GET Service Engine Related Configuration
print("\033[1mExtracting TARGET Services Engine Group information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
print()

segroup_data = target_segroup_data
# Define GET request parameters
se_group_uuid=segroup_data["uuid"]
url_path = "serviceengine"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}"
}

# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)
 
# Save Result
target_se_data = resp_data

In [ ]:
# Exploring Interface Information of SOURCE SE

for i in range(len(target_se_data)):
    print("\033[1mShowing information of the TARGET SE "+str(i+1)+" \033[0m")
    print("\033[1m--------------------------------------------------\033[0m")
    se_data_vnics = target_se_data[i]["data_vnics"]
    se_df = pd.DataFrame(se_data_vnics)
    display(se_df)
    time.sleep(1/4)

### Select one FREE interface at Target Service Engines that interface that will be used to receive gathered ip address configuration from source interface at source Service Engines


In [ ]:
target_if_name = "eth2"

In [ ]:
# Assuming IP Address will be REUSED!!!
#  1.- Migrate VS (change SEGroup)
#  2.- Migrate SEs interfaces
#  3.- Migrate Network Services (if exists)
# Disable MIGRATING SE INTERFACES TO AVOID ITF CLASHING

## 2.1  Migrate Service Engine Interfaces

In [ ]:
# Display gathered information prior to execute next cell


print ("The following interfaces has been selected for migration")
print()
for i in range(len(output_config)):
    print("For the SOURCE Service Engine "+str(i+1)+" with name \033[1m" + source_se_data[i]["name"]+"\033[0m")
    print("The interface \033[1m" + source_if_name + "\033[0m has been selected with following configuration")
    print("   IP Address \033[1m" + output_config[i]["ip_addr"] +"\033[0m")
    print("   Mask Length \033[1m" + str(output_config[i]["mask"]) +"\033[0m")
    print()
print("---------------------")
for i in range(len(target_se_data)):
    print("For the TARGET Service Engine "+str(i+1)+" with name \033[1m" + target_se_data[i]["name"]+"\033[0m")
    print("The interface \033[1m" + target_if_name + "\033[0m will be configured with above information.")
    print("   IP Address \033[1m" + output_config[i]["ip_addr"] +"\033[0m")
    print("   Mask Length \033[1m" + str(output_config[i]["mask"]) +"\033[0m")
    print()
print("---------------------")
print("WARNING!!! Before proceeding migrated interfaces in the source SEs will be disabled")

In [ ]:
# Create dictionary list containing pair of migrating-target SEs  
se_pairs =[
     {"source_se_uuid": source_se_data[0]["uuid"],
      "target_se_uuid": target_se_data[0]["uuid"]
     },
     {"source_se_uuid": source_se_data[1]["uuid"],
     "target_se_uuid": target_se_data[1]["uuid"]
     }
]
# Loop to migrate 2 se Pairs (migrating to target) 
for i in range(len(se_pairs)):
    source_se_uuid = se_pairs[i]["source_se_uuid"]
    target_se_uuid = se_pairs[i]["target_se_uuid"]
    print("Migrating pair "+str(i+1))
    print("-------------------------")
    print("Migrating (source) Service Engine UUID \033[1m"+source_se_uuid +"\033[0m at SE Group \033[1m"+source_segroup+"\033[0m") 
    print("Receiving (target) Service Engine UUID \033[1m"+target_se_uuid+"\033[0m at SE Group \033[1m"+target_segroup+"\033[0m")
    source_se_uuid = se_pairs[i]["source_se_uuid"]
    target_se_uuid = se_pairs[i]["target_se_uuid"]

    # Disabling interface to migrate at source SE

    # Extract the source/target SE data with matching uuid
    source_se_json_data = [ se for se in source_se_data if se.get("uuid") == source_se_uuid]
    target_se_json_data = [ se for se in target_se_data if se.get("uuid") == target_se_uuid]


    # Extract source_interface vnic_networks object
    json_data_source_adapter = [ data_vnic for data_vnic in source_se_json_data[0]["data_vnics"] if data_vnic.get("if_name") == source_if_name ]
    json_data_source_adapter_vnic_networks = [json_data_source_adapter[0]["vnic_networks"][0]]

    # Modifying existing value for selected interface at source SE (i.e Enabled = False for corresponding interface to shutdown )
    # vnic_network list must be removed to avoid SE interface overlapping
    if_name_key = "if_name"
    if_name_value = source_if_name

    # Loop through each dictionary in the list
    for adapter in source_se_json_data[0]["data_vnics"]:
        # Check if the key_to_check matches the value_to_match
        if adapter.get(if_name_key) == if_name_value:
            # Update the value of key_to_modify
            adapter["enabled"] = False
            adapter["vnic_networks"]=[]
    
    # Remove _last_modified key
    source_se_json_data[0].pop("_last_modified", None)

    # Disable migrating interfaces to avoid IP address overlapping
    body = source_se_json_data[0]
    print ("Applying changes to source SE "+str(i+1)+" with uuid "+ source_se_uuid)
    print (" - Changing interface "+source_if_name+" to disabled state")
    print (" - Removing IP Address Configuration")

    url_path = "serviceengine/"+source_se_uuid
    resp = api.put (url_path, data=json.dumps(body))

    if resp.status_code in range(200, 299):
      print(resp)
      print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
      print()
    else:
      print('Error in modifying '+url_path+' :%s' % resp.text)

    # PUT to configure migrating IP Addresses in target SE
    # Extract IP Address from migrating SE
    # Importing extracted information into target body
    if_name_key = "if_name"
    if_name_value = target_if_name

    # Loop through each dictionary in the list
    for item in target_se_json_data[0]["data_vnics"]:
      # Check if the interface name correspondes to given value
      if item.get(if_name_key) == if_name_value:
        # Update the value of vnic_networks from source SE
        item["vnic_networks"] = json_data_source_adapter_vnic_networks
    
    # Remove _last_modified key
    target_se_json_data[0].pop("_last_modified", None)

    body = target_se_json_data[0]
    print ("Applying changes to target SE "+str(i+1)+" with uuid "+ target_se_uuid)
    print (" - Applying IP Address Configuration")
    print(target_se_json_data[0])

    url_path = "serviceengine/"+target_se_uuid
    resp = api.put (url_path, data=json.dumps(body))

    if resp.status_code in range(200, 299):
      print(resp)
      print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
      print()
    else:
      print('Error in modifying '+url_path+' :%s' % resp.text)

## 2.1  Migrate Virtual Services

In [ ]:
# Increase the number of VS per SE to acommodate imported VS
target_max_vs_per_se = target_segroup_data["max_vs_per_se"]
count_of_source_imported_vs = len(source_vs_data)
count_of_current_target_vs = len(target_vs_data)
if ((count_of_source_imported_vs + count_of_current_target_vs) >= (target_max_vs_per_se - 5)):
    target_segroup_data["max_vs_per_se"] = count_of_source_imported_vs + count_of_current_target_vs + 5
    print ("Increase the number of VS per SE to "+str(target_segroup_data["max_vs_per_se"])+" to acommodate imported VSs ")
    body = target_segroup_data
    # Remove _last_modified key to avoid concurrent update error
    body.pop("_last_modified", None)

    url_path = "serviceenginegroup/"+target_segroup_data["uuid"]
    resp = api.put (url_path, data=json.dumps(body))

    if resp.status_code in range(200, 299):
       print(resp)
       print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
       print()
    else:
       print('Error in modifying '+url_path+' :%s' % resp.text)

In [ ]:
# Obtain the target se_group_ref (corresponds to key url of the serviceenginegroup object ) 
target_se_group_ref = target_segroup_data["url"]

for item in source_vs_data:
    item["se_group_ref"]=target_se_group_ref
    print ("Service engine group reference has been changed for Virtual Service "+ item["name"])
    print ("Applying new configuration")

    # Remove _last_modified key to avoide concurrent update error
    item.pop("_last_modified", None)

    url_path = "virtualservice/"+item["uuid"]
    resp = api.put (url_path, data=json.dumps(item))

    if resp.status_code in range(200, 299):
       print(resp)
       print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
       print()
    else:
       print('Error in modifying '+url_path+' :%s' % resp.text)